# <strong>Evaluate your Five Flags and Citation Profile plots</strong>

This program takes as Input your citations to each of your papers, in any order and all values, and generates as Output, your $5F$ and Hirsch curve. Your Hirsch curve is a histogram of citations $c(s)$ to papers ranked serially as $s=1,2... N_p$, arranged in decreasing order. The <strong>p</strong>runed number of papers $N_p$ are those cited more than once. Thus citation values retained are in the range $C_{\text{max}} \equiv c(1) \geq c(s) \geq C_{\text{min}} \equiv  c(N_p) > 1$.  

The Output Citation Profile plots are:
-  The Hirsch curve or linear-linear plot of citations $c(s)$ versus rank of the papers $s$.
- The log-log plot of citations $c(s)$ versus  rank of the papers $s$.
- The linear-linear  plot with the origin taken as the $H$-point $(h, c(h))$, or scaled plot of $Y\equiv[c(s)-c(h)]/c(h)$ versus $x\equiv[s-h]/h$. The dashed line has slope $-1$.

The Output also delivers your five flags or $5F$:
- Your $h$-index or number $s=h$ of higher-cited papers with $c(h) \geq h$.
- Your <strong>u</strong>ppermost or maximum citation scaled in $h$ or $u \equiv C_{\text{max}}/h$.
- Your fraction of higher-cited or $h$-type papers $r \equiv h/N_p $, with remaining fraction of  normally-cited or $n$-type papers  as $1-r$.
- Your $nac$ number = <strong>n</strong>-type <strong>a</strong>verage <strong>c</strong>itation over this larger fraction $1-r$, or the normally-cited papers.
- Your $hac$ number = <strong>h</strong>-type <strong>a</strong>verage <strong>c</strong>itation over the smaller fraction $r$, or the higher-cited papers. 

The Output also delivers useful quantities (that may also be obtained from the $5F$):
- Total papers $N_p$ with citations $>1$.
- Total citations over $A$ years,  $N_{c,\text{tot}} \equiv \sum_{s=1}^{N_p} c(s)$ citations.
- Average citations per paper, over $A$ years, $\bar{C}(A)\equiv N_{c,\text{tot}}/N_p$ citations.
- Rate of publication $R_p \equiv N_p/A$ papers per year.
- Rate of incremental citation, $R_c \equiv N_{c,\text{tot}}/(A~N_p)$ citations per paper, per year.
- Maximum citation $C_{max}\equiv c(1)$ citations.
- Deviation of the $H$-point from the diagonal as a fraction  $\rho(h) \equiv 1- h/c(h)$.

### <strong>Instructions:</strong>
1. On your computer prepare a data file of citations, written in a single column. Save this file as YourName.dat (keep '.dat' extension). 
      - Click <a href="your-data/F8.dat" download="F8.dat">here</a> to see 'F8.dat' example data file that can be used in the procedure.
2. Upload your data file YourName.dat using the 'Upload' button.
      - The selected data file name should be visible in the box below.
3. Write your name in the input box below.
3. Enter your academic age in years (integer). $A= \text{present year}-\text{year of first publication +1}$.
4. Click 'Evaluate' button to evaluate your Five Flags and Citation Profile plots.
5. Click the 'Download' button to get your Output in a pdf file.
6. In case of difficulty, kindly message pnkjpopli@gmail.com.

<sub><sup>(* This application and its source code is distributed under creative commons <a href="https://creativecommons.org/licenses/" target="_blank">[CC BY-NC-SA 4.0]</a> license. To see the source code of this application visit <a href="https://github.com/pankajpopli/cit-prof.git" target="_blank">github-repo</a>.
<br>In short, this license lets you use, remix, adapt, and build upon the work non-commercially, as long as you give credit and license new creations under the identical terms.*)</sup></sub>


In [ ]:
import ipywidgets as widgets
from ipywidgets import HTML
from termcolor import colored
from IPython.display import display, Markdown

import base64
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import rc
import numpy as np
from fpdf import FPDF
from PyPDF2 import PdfFileMerger, PdfFileReader  
plt.style.use('./settings/matplotlibrc')


out1=widgets.Output()
out2=widgets.Output()
out3=widgets.Output()


In_name='Your Name'
In_fileName='ExampleData.dat'
In_age=1

Dwnldfile='Name'
payload = None
encoded_string=None
hIndex,r,u,nac,hac,Np,Rp,Rc,Nctot,Cbar,Cmax=(1,)*11
def calculate_profile(fname,fage,file_name):
    with out1:
        global hIndex,r,u,nac,hac,Np,Rp,Rc,Nctot,Cbar,Cmax
        file_path = fr'your-data/{file_name}'
        csd = np.loadtxt(file_path,unpack=True)
        Impdata = csd[csd>=2];
        Np=Impdata.size;

        if Np == 0:
            print(colored('\n--------------------------------','red'))
            print(colored('     Insufficient data','red'))
            print(colored('--------------------------------\n','red'))
            print(colored('\nOnly citations >= 2  are considered for evaluation.','red'))
            print(colored('\nNo paper with more than 2 citation is found in provided data.','red'))
            print(colored('\nAborting.','red'))
            return

        SortedData = np.c_[ np.arange(1,Np+1,1),-np.sort(-Impdata) ]
        hIndex=(SortedData[SortedData[:,1]>=SortedData[:,0]])[-1,0]

        r = hIndex/Np;
        Nctot=np.sum(SortedData[:,1])
        Cbar = (Nctot/Np)
        Cmax=np.max(SortedData[:,1])
        CofNp = SortedData[-1,1]
        Cofh=(SortedData[SortedData[:,1]>=SortedData[:,0]])[-1,1]
        hac=(SortedData[0:int(hIndex), 1].sum())/(hIndex)
        
        u=Cmax/hIndex
        rho=(1-(hIndex/Cofh))
        Rp= Np/fage
        Rc= Nctot/(Np*fage)

        if Np<=hIndex:
            print(colored('\n--------------------------------','red'))
            print(colored('     Insufficient data','red'))
            print(colored('--------------------------------\n','red'))
            print(colored('\nTotal number of papers Np should at least be greater than the h-index to calculate all five Flags.','red'))
            print(colored('\nOnly Np = {} papers with more than two citations found in the provided data.','red').format(Np))
            print(colored('\nh-index is {}.','red').format(hIndex))
            print(colored('\nCannot calculate "nac". Below are some other calculated quantities','red'))
            print(colored('\nOther Flags are.','red'))
            print(colored("\nFive Flags:\n h = %d\n r = %.2f\n u = %.2f\n hac = %.2f\n"%(hIndex,r,u,hac),'red'))
            print(colored("Other derived quantities and more:\n Np = %d\n Rp = %.2f\n Rc = %.2f\n Nctot = %d\n c_bar(A) = %.2f\n c_max = %d\n c(h) = %d\n rho(h) = %.2f\n c(Np) = %d"%(Np,Rp,Rc,Nctot,Cbar,Cmax,Cofh,rho, CofNp),'red'))
            print(colored('\nAborting.','red'))
            return
        else:
            nac=(SortedData[int(hIndex):, 1].sum())/(Np-hIndex)

        #plot limits
        lx1 = 1
        lx2 = Np
        ly1 = 1
        ly2 = Cmax

        # seting up canvas
        fig0 = plt.figure(constrained_layout=True,figsize=(45,15))
        gs =  mpl.gridspec.GridSpec(nrows=3,ncols=1,figure=fig0)
        axes0 = fig0.add_subplot(gs[0],box_aspect=1)
        axes1 = fig0.add_subplot(gs[1],box_aspect=1)
        axes2 = fig0.add_subplot(gs[2],box_aspect=1)
        fig0.set_constrained_layout_pads(hspace=0.07)
        

        fig1 = plt.figure(figsize=(5,5))
        ax1 = plt.axes()
        
        fig2 = plt.figure(figsize=(5,5))
        ax2 = plt.axes()
        
        fig3 = plt.figure(figsize=(5,5))
        ax3 = plt.axes()

        # axis limits and label lin-lin
        ax1.set_xlim([0,lx2+10]) 
        ax1.set_ylim([0,ly2+50]) 
        ax1.set_ylabel(r'c(s)')
        ax1.set_xlabel(r's',labelpad=-5)
        ax1.tick_params(axis='both', which='major', labelsize=19)
        # ax1.set_title(fr'{fname}',fontsize=20)
        axes0.set_xlim([0,lx2+10]) 
        axes0.set_ylim([0,ly2+50]) 
        axes0.set_ylabel(r'c(s)')
        axes0.set_xlabel(r's',labelpad=-5)
        axes0.tick_params(axis='both', which='major', labelsize=19)

        # axis limits and label log-log
        ax2.set_xscale('log')
        ax2.set_yscale('log')
        ax2.set_xlim([lx1,lx2+5]) 
        ax2.set_ylim([ly1,ly2+50]) 
        #ax2.set_title(fr'Citation Profile: {fname}',fontsize=20)
        ax2.set_ylabel(r'c(s)')
        ax2.set_xlabel(r's',labelpad=-5)
        ax2.tick_params(axis='both', which='major', labelsize=19)
        axes1.set_xscale('log')
        axes1.set_yscale('log')
        axes1.set_xlim([lx1,lx2+5]) 
        axes1.set_ylim([ly1,ly2+50]) 
        axes1.set_ylabel(r'c(s)')
        axes1.set_xlabel(r's',labelpad=-5)
        axes1.tick_params(axis='both', which='major', labelsize=19)

        # axis limits and label scaled
        ax3.set_xlim([-1,3]) 
        ax3.set_ylim([-1,3])
        ax3.set_ylabel(fr'$Y$')
        ax3.set_xlabel(r'$x$',labelpad=-5)
        ax3.tick_params(axis='both', which='major', labelsize=19)
        axes2.set_xlim([-1,3]) 
        axes2.set_ylim([-1,3])
        axes2.set_ylabel(fr'$Y$')
        axes2.set_xlabel(r'$x$',labelpad=-5)
        axes2.tick_params(axis='both', which='major', labelsize=19)

        #lin-lin
        ax1.plot(SortedData[:,0],SortedData[:,1],"o-",ms=6, color=(0, 0.8, 0.8, 1),markeredgecolor=(0,0,0,1),markeredgewidth=0.3)
        ax1.plot(hIndex, Cofh, '+',ms=25,c='black',markeredgewidth=1.2)
        ax1.annotate(r'$h$', xy=(hIndex, Cofh), fontsize=20, xycoords='data',xytext=(5, 5), textcoords='offset points',fontweight='ultralight')
        plt.close(fig1)

        axes0.plot(SortedData[:,0],SortedData[:,1],"o-",ms=6, color=(0, 0.8, 0.8, 1),markeredgecolor=(0,0,0,1),markeredgewidth=0.3)
        axes0.plot(hIndex, Cofh, '+',ms=25,c='black',markeredgewidth=1.2)
        axes0.annotate(r'$h$', xy=(hIndex, Cofh), fontsize=20, xycoords='data',xytext=(5, 5), textcoords='offset points',fontweight='ultralight')

        #log-log
        ax2.plot(SortedData[:,0],SortedData[:,1],"o-",ms=6, color=(0, 0.8, 0.8, 1),markeredgecolor=(0,0,0,1),markeredgewidth=0.3)
        ax2.plot(hIndex, Cofh, '+',ms=25,c='black',markeredgewidth=1.2)
        ax2.annotate(r'$h$', xy=(hIndex, Cofh), fontsize=20, xycoords='data',xytext=(5, 5), textcoords='offset points',fontweight='ultralight')
        plt.close(fig2)

        axes1.plot(SortedData[:,0],SortedData[:,1],"o-",ms=6, color=(0, 0.8, 0.8, 1),markeredgecolor=(0,0,0,1),markeredgewidth=0.3)
        axes1.plot(hIndex, Cofh, '+',ms=25,c='black',markeredgewidth=1.2)
        axes1.annotate(r'$h$', xy=(hIndex, Cofh), fontsize=20, xycoords='data',xytext=(5, 5), textcoords='offset points',fontweight='ultralight')

        #scaled
        line_x = np.linspace(-1,1,100)
        line_x2 = np.linspace(-20,1+1/hIndex,100)
        ax3.plot(((SortedData[:,0]/hIndex)-1),((SortedData[:,1]/Cofh)-1),"o-",ms=6, color=(0, 0.8, 0.8, 1),markeredgecolor=(0,0,0,1),markeredgewidth=0.3)
        ax3.plot(line_x,-line_x,"--",color='Black',linewidth=1)
        ax3.plot(line_x2,line_x2,":",color='Black',linewidth=1)
        ax3.plot([-20,20], [0, 0],color='k', linestyle='-', linewidth=1.0,alpha=0.5);
        ax3.plot([0,0], [-20, 20],color='k', linestyle='-', linewidth=1.0,alpha=0.5);
        plt.close(fig3)

        axes2.plot(((SortedData[:,0]/hIndex)-1),((SortedData[:,1]/Cofh)-1),"o-",ms=6, color=(0, 0.8, 0.8, 1),markeredgecolor=(0,0,0,1),markeredgewidth=0.3)
        axes2.plot(line_x,-line_x,"--",color='Black',linewidth=1)
        axes2.plot(line_x2,line_x2,":",color='Black',linewidth=1)
        axes2.plot([-20,20], [0, 0],color='k', linestyle='-', linewidth=1.0,alpha=0.5);
        axes2.plot([0,0], [-20, 20],color='k', linestyle='-', linewidth=1.0,alpha=0.5);

        # ax3.plot(hIndex, Cofh, '+',ms=25,c='black',markeredgewidth=1.2)
        # ax3.annotate(r'$h$', xy=(hIndex, Cofh), fontsize=20, xycoords='data',xytext=(5, 5), textcoords='offset points',fontweight='ultralight')

        props = dict(boxstyle='round', linewidth=1,facecolor=(0,0.6,0.63,0.1))
        textstr = '\n'.join((r'$h=%d$' % (hIndex, ),r'$r=%.2f$' % (r,),r'$u=%.2f$' % (u,),r'$\mathrm{nac}=%.2f$' % (nac,),r'$\mathrm{hac}=%.2f$' % (hac,)  )  )
        ax1.text(0.95, 0.68, textstr,ha='right', transform=ax1.transAxes, fontsize=14,bbox=props,fontweight='ultralight')
        axes0.text(0.95, 0.69, textstr,ha='right', transform=axes0.transAxes, fontsize=14,bbox=props,fontweight='ultralight')
        ax2.text(0.06,0.06, textstr,ha='left', transform=ax2.transAxes, fontsize=14,bbox=props,fontweight='ultralight')
        axes1.text(0.06,0.06, textstr,ha='left', transform=axes1.transAxes, fontsize=14,bbox=props,fontweight='ultralight')
        print("\nCitation Profile for %s with age = %d"%(fname,fage));
        print(colored('\n(all numbers rounded up to two decimal places)','blue'))
        print(colored('---------------------------------------------------','blue'))
        print(colored('Total number of items detected = ','blue'), colored(csd.size,'blue'))
        print(colored('\nTotal number of items after removing ephemera = ','blue'),colored(Np,'blue'))
        print(colored("\nFive Flags:\n h = %d\n r = %.2f\n u = %.2f\n nac = %.2f\n hac = %.2f\n"%(hIndex,r,u,nac,hac),'blue'))
        print(colored("Other derived quantities and more:\n Np = %d\n Nc,tot = %d\n C_bar(A) = %.2f\n Rp = %.2f\n Rc = %.2f\n C_max = %d\n rho(h) = %.2f\n c(h) = %d\n C_min = c(Np) = %d"%(Np,Nctot,Cbar,Rp,Rc,Cmax,rho,Cofh,CofNp),'blue'))
        print(colored('---------------------------------------------------','blue'))
        print(colored("\n\nClick the Download button to get results in a pdf file.\n",'blue'))
        fig1.savefig(fr'your-results/lin-lin-cit-prof_{fname}.pdf');
        fig2.savefig(fr'your-results/log-log-cit-prof_{fname}.pdf');
        fig3.savefig(fr'your-results/scaled-cit-prof_{fname}.pdf');
        # fig3.savefig(fr'your-results/cit-prof_{fname}.pdf');
        plt.close()
        
        fw = open(fr'your-results/data-{fname}.txt', "w")
        fw.write("\nCitation Profile for %s with age = %d\n"%(In_name,In_age));
        fw.write("\n\n(all numbers rounded up to two decimal places)\n");
        fw.write("---------------------------------------------------------------------");
        fw.write("\nTotal number of items detected = %d"%csd.size)
        fw.write("\nTotal number of items after removing ephemera = %d\n"%Np)
        fw.write("\n\nFive Flags:\n h = %d\n r = %.2f\n u = %.2f\n nac = %.2f\n hac = %.2f\n\n"%(hIndex,r,u,nac,hac))
        fw.write("\n\nOther derived quantities and more:\n Np = %d\n Nc,tot = %d\n C_bar(A)= %.2f\n Rp = %.2f\n Rc = %.2f\n C_max = %d\n rho(h) = %.2f\n c(h) = %d\n C_min = c(Np) = %d"%(Np,Nctot,Cbar,Rp,Rc,Cmax,rho,Cofh,CofNp))
        fw.write("\n---------------------------------------------------------------------");
        fw.write("\n\nYou can freely use all the above generated data.")
        fw.write("\nContact pnkjpopli@gmail.com in case of any error.")
        fw.close()
        # writing data to a txt file and making pdf
        pdf = FPDF();  
        # Add a page
        pdf.add_page()
        pdf.set_font("Times", size = 15)
        with open(fr'your-results/data-{fname}.txt', encoding='utf-8') as txtf:
            contents = txtf.read()
        pdf.multi_cell(0,6.5, align='L', txt=contents, border=0)
        pdf.output(fr'your-results/data-{fname}.pdf') 

        # merging citation-plot and data pdf file
        merger = PdfFileMerger()
        merger.append(PdfFileReader(open(fr'your-results/log-log-cit-prof_{fname}.pdf', 'rb')))
        merger.append(PdfFileReader(open(fr'your-results/lin-lin-cit-prof_{fname}.pdf', 'rb')))
        merger.append(PdfFileReader(open(fr'your-results/scaled-cit-prof_{fname}.pdf', 'rb')))
        merger.append(PdfFileReader(open(fr'your-results/data-{fname}.pdf', 'rb')))
        merger.write(fr'your-results/cit-prof_{fname}.pdf')
        
        
        
        
        
        global Dwnldfile
        global payload
        global html_button
        Dwnldfile = fr"cit-prof_{fname}.pdf"
        with open(fr'your-results/{Dwnldfile}', "rb") as pdf_file:
            encoded_string = base64.b64encode(pdf_file.read())
        payload = encoded_string.decode()
        html_button = html_buttons.format(payload=payload,filename=Dwnldfile)
        with out2:
            display(fig0)
        with out3:
            display(HTML(html_button))           
            
            
# Create widget
def on_upload_change(change):
    if change.new==0:
        return
    up = change.owner
    global In_fileName
    for filename,data in up.value.items():
        In_fileName=filename
        disp_text.value=fr'selected file: [ {In_fileName} ]'
        with open(fr'your-data/{In_fileName}', 'wb') as f:
            f.write(data["content"])
        f.close()
    up.value.clear()
    up._counter = 0



def on_age_change(change):
    global In_age
    if change['type'] == 'change' and change['name'] == 'value':
        In_age = int(change['new'])
            
def on_name_change(change):
    global In_name
    if change['type'] == 'change' and change['name'] == 'value':
        In_name = change['new']

def on_fname_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        disp_text.value = change['new']
            
            
def on_evaluate_button_clicked(x):
    out1.clear_output(wait=True)
    out2.clear_output(wait=True)
    out3.clear_output(wait=True)
    calculate_profile(In_name,In_age,In_fileName)


html_buttons = '''<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<a download="{filename}" href="data:pdf;base64,{payload}" download>
<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning">Download File</button>
</a>
</body>
</html>
'''

upload_btn = widgets.FileUpload(accept='.dat')
input_name = widgets.Text(value='Your Name',description='Name:',disabled=False)
input_age =  widgets.BoundedIntText(value=1,min=1,description='Acad. age:',disabled=False)
evaluate_bttn = widgets.Button(description="Evaluate")
disp_text = widgets.Text(value=fr'selected file: [ {In_fileName} ]',disabled=True)


upload_btn = widgets.FileUpload(accept='.dat')
upload_btn.observe(on_upload_change, names='_counter')
input_age.observe(on_age_change)
input_name.observe(on_name_change)
disp_text.observe(on_upload_change,names='_counter')
evaluate_bttn.on_click(on_evaluate_button_clicked)


    

#app layout
header = widgets.HTML("<h1>Citation Profiler:</h1>", layout=widgets.Layout(height='auto'))
# footer = widgets.HTML("<hr style='height:2px;'>",layout=widgets.Layout(height='auto') )


left_box = widgets.VBox([widgets.Label("File upload and details:"),
                             upload_btn,disp_text,
                            input_name,
                            input_age,
                            #widgets.Label("*Age: Number of years since your first publication."),
                             widgets.Label("Evaluate:"),
                             evaluate_bttn,out1,out3] )
right_box =  out2
display(widgets.Output(layout={'border': '2px solid black'}))
display(widgets.Output(layout={'border': '2px solid black'}))
widgets.AppLayout(header=header,
        left_sidebar=left_box,
        center=None,
        right_sidebar=right_box,
        footer=None,
        pane_widths=[3, 0, 3],
        pane_heights=[1, 20, '10px'],
        border='solid'
        )


#use binder link like this
#https://mybinder.org/v2/gh/pankajpopli/cit-prof/main?urlpath=apps%2FCalculate-cit-prof.ipynb 